In [11]:
import pandas as pd 
import json
from arcgis.features import FeatureLayer

In [12]:
# url = 'https://services1.arcgis.com/79kfd2K6fskCAkyg/arcgis/rest/services/FoodServiceData/FeatureServer/0'

# batch_size = 1000  # Number of records to retrieve per batch
# offset = 0  # Initial offset value
# data_list = []
# # ​Create the feature layer object
# feature_layer = FeatureLayer(url)

# while True:
#     # Query the feature layer with pagination
#     query_result = feature_layer.query(where='1=1', out_fields='*', return_geometry=False, result_offset=offset, result_record_count=batch_size)
    
#     # Retrieve the features from the query result
#     features = query_result.features
    
#     # Process the data for the current batch
#     for feature in features:
#         data_list.append(feature.attributes)
    
#     # Break the loop if the response is empty or the desired number of records is reached
#     if len(features) == 0 or len(data_list) >= 1000:
#         break
    
#     # Increment the offset by the batch size
#     offset += batch_size
# # Create a DataFrame from the data list
# df = pd.DataFrame(data_list)

In [13]:
df = pd.read_csv("json_files/local_working.csv")

In [14]:
cols_drop = ['InspectionID', 'PlaceName', 'Address2', 'TypeDescription', 'NameSearch', 'Intersection']
df.drop(cols_drop, axis=1, inplace=True)

In [15]:
df['InspectionDate'] = pd.to_datetime(df['InspectionDate'])

In [16]:
df.head()

,EstablishmentID,Ins_TypeDesc,EstablishmentName,Address,City,State,Zip,InspectionDate,score,Grade,ObjectId
0,98505,REGULAR,MI SUENO RESTAURANT,3425 BARDSTOWN RD,LOUISVILLE,KY,40218,2023-04-18,70,C,4803
1,118055,REGULAR,DAISUKI SUSHI,1604 BARDSTOWN RD,LOUISVILLE,KY,40205,2023-04-14,71,C,10803
2,99781,REGULAR,HAPPY CHINA,9128 TAYLORSVILLE RD,LOUISVILLE,KY,40299,2022-12-22,72,C,5049
3,115438,FOLLOWUP,EXPRESSWAY FOOD MART,1420 W BROADWAY,LOUISVILLE,KY,40203,2022-12-27,73,C,9916
4,107689,REGULAR,BEST WESTERN,4110 DIXIE HWY,LOUISVILLE,KY,40216,2022-09-28,74,C,6908


In [17]:
# Grouping by EstablishmentID and aggregating the necessary columns
grouped_df = df.groupby("EstablishmentID").agg({
    "EstablishmentName": "first",
    "Address": "first",
    "score": lambda x: list(x)
})

# Sorting the establishments based on the count of low scores
grouped_df["low_score_count"] = grouped_df["score"].apply(lambda x: sum(score < 75 for score in x))
grouped_df = grouped_df.sort_values("low_score_count", ascending=False)

# Selecting the top 20 establishments with the most low scores
top_20_establishments = grouped_df.head(20)

# Creating the final DataFrame with the desired columns
final_df = top_20_establishments[["EstablishmentName", "Address", "score"]]

# Printing the final DataFrame
final_df.head(30)

,EstablishmentName,Address,score
EstablishmentID,,,
118055,DAISUKI SUSHI,1604 BARDSTOWN RD,"[71, 95, 100]"
115438,EXPRESSWAY FOOD MART,1420 W BROADWAY,"[73, 78, 91, 93, 93, 96, 98]"
35239,AL WATAN BAKERY,3711 KLONDIKE LN,"[74, 90]"
98505,MI SUENO RESTAURANT,3425 BARDSTOWN RD,"[70, 80, 96, 97]"
92904,DIXIE CHICKEN,3947 DIXIE HWY,"[74, 93, 96, 98]"
109770,MASHALLAH AFRICAN RESTAURANT,4109 BARDSTOWN RD STE 104,"[74, 94, 99]"
107689,BEST WESTERN,4110 DIXIE HWY,"[74, 93, 97]"
99781,HAPPY CHINA,9128 TAYLORSVILLE RD,"[72, 91, 97]"
109006,ENRIQUES PLACE #2,None,[98]


In [18]:
# Grouping by EstablishmentID and calculating average scores
avg_scores_df = df.groupby("EstablishmentID").agg({
    "EstablishmentName": "first",
    "Address": "first",
    "score": "mean"
})

# Sorting the establishments based on average scores
sorted_restaurants = avg_scores_df.sort_values("score", ascending=True)

# Displaying the restaurants with consistently lower scores
sorted_restaurants.head(40)



,EstablishmentName,Address,score
EstablishmentID,,,
35239,AL WATAN BAKERY,3711 KLONDIKE LN,82.000000
98505,MI SUENO RESTAURANT,3425 BARDSTOWN RD,85.750000
112909,TAQUERIA LOS GORDITOS,2017 BROWNSBORO RD STE B,86.000000
107692,FAMILY TIES,1030 CECIL AVE,86.333333
99781,HAPPY CHINA,9128 TAYLORSVILLE RD,86.666667
107689,BEST WESTERN,4110 DIXIE HWY,88.000000
35038,STRIKE & SPARE FAMILY FUN CENTER,6220 SHEPHERDSVILLE RD,88.000000
81788,WINGS STATION,2119 CRUMS LN,88.000000
118055,DAISUKI SUSHI,1604 BARDSTOWN RD,88.666667
